### Looks into total training time and time per epoch
Should probably have done this in test analysis so I could include the results in that table
but I didnt...

In [8]:
import os
import pandas as pd
import numpy as np

In [9]:
source_dir = "/home/vincent/Documenten/BachelorsProject/ft_artist"
target_dir = "/home/vincent/Documenten/BachelorsProject/GitHub_Repo/results/ft_artist"

models = {
    "vit_b_16": {"vit": True},
    "swin_b": {"vit": True},
    "beit_b_16": {"vit": True},
    "deit_b_16": {"vit": True},
    "vgg19": {"vit": False},
    "resnet50": {"vit": False},
    "efficientnetv2_m": {"vit": False},
    "convnext_b": {"vit": False}
}

In [10]:
def getDuration(timefile: str):
    """" Extracts the training time in seconds from the time file """
    with open(timefile) as f:
        tmp = f.read()
    prefix = "Training time: "
    return float(tmp[len(prefix):])

In [11]:
# For each model and dataset we add a [training_time, time_per_epoch] pair
for model in models:
    models[model]["pairs"] = []

for file in os.scandir(source_dir):
    if file.is_file() and "validation" in file.name:
        timefile = file.path.replace("validation.csv", "trainingtime.txt")
        if os.path.exists(timefile):
            for model in models:
                if model not in file.name:
                    continue
                epochs = len(pd.read_csv(file.path))
                duration = getDuration(timefile)
                models[model]["pairs"] += [[duration, duration / epochs]]
        else:
            print("ERROR! {timefile} did not exist!")
            break

In [12]:
for model in models:
    models[model]["time_mean"] = np.mean([x[0] for x in models[model]["pairs"]])
    models[model]["time_std"] = np.std([x[0] for x in models[model]["pairs"]])
    models[model]["time_median"] = np.median([x[0] for x in models[model]["pairs"]])
    models[model]["time_q1"] = np.quantile([x[0] for x in models[model]["pairs"]], q=0.25)
    models[model]["time_q3"] = np.quantile([x[0] for x in models[model]["pairs"]], q=0.75)

    models[model]["epoch_time_mean"] = np.mean([x[1] for x in models[model]["pairs"]])
    models[model]["epoch_time_std"] = np.std([x[1] for x in models[model]["pairs"]])
    models[model]["epoch_time_median"] = np.median([x[1] for x in models[model]["pairs"]])
    models[model]["epoch_time_q1"] = np.quantile([x[1] for x in models[model]["pairs"]], q=0.25)
    models[model]["epoch_time_q3"] = np.quantile([x[1] for x in models[model]["pairs"]], q=0.75)

In [13]:
table = []
for model in models:
    table += [[
        model,
        models[model]["time_mean"],
        models[model]["time_std"],
        models[model]["time_median"],
        models[model]["time_q1"],
        models[model]["time_q3"],
        models[model]["epoch_time_mean"],
        models[model]["epoch_time_std"],
        models[model]["epoch_time_median"],
        models[model]["epoch_time_q1"],
        models[model]["epoch_time_q3"]
    ]]

df = pd.DataFrame(table, columns=[
        "model",
        "time_mean",
        "time_std",
        "time_median",
        "time_q1",
        "time_q3",
        "epoch_time_mean",
        "epoch_time_std",
        "epoch_time_median",
        "epoch_time_q1",
        "epoch_time_q3"
])

df

,model,time_mean,time_std,time_median,time_q1,time_q3,epoch_time_mean,epoch_time_std,epoch_time_median,epoch_time_q1,epoch_time_q3
0,vit_b_16,626.211769,117.652114,572.116752,539.349504,747.136945,28.697901,0.215561,28.708635,28.605838,28.736036
1,swin_b,947.664664,144.743174,880.374794,841.133526,1043.974283,39.832782,0.302403,40.017036,39.536044,40.053977
2,beit_b_16,851.106010,101.683111,829.367493,799.335423,960.170183,30.613283,0.263487,30.717315,30.397039,30.743670
3,deit_b_16,899.319360,218.747163,779.642490,776.308930,912.354488,27.602865,0.179514,27.647106,27.434331,27.725319
4,vgg19,762.143520,288.879361,665.367546,561.205899,698.934347,20.700801,0.099319,20.763447,20.605527,20.785404
5,resnet50,533.455392,93.976271,572.026123,483.705658,594.503376,14.995293,0.093542,15.032379,14.912380,15.053319
6,efficientnetv2_m,958.388712,280.184500,970.649410,689.096171,1045.148154,38.627097,0.183259,38.684321,38.632875,38.709191
7,convnext_b,1091.987370,347.587924,1014.762010,881.084667,1021.820031,46.233577,0.235300,46.372877,46.125546,46.403691


In [14]:
df.to_csv(os.path.join(target_dir, "durations.csv"), index=False)